<a href="https://colab.research.google.com/github/YoshiakiAsahara/for-intern/blob/main/%E3%82%A4%E3%83%B3%E3%82%BF%E3%83%BC%E3%83%B3%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [ ]:
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler


if 'PG_PORT' in os.environ:

    host = 'db'
    port = os.environ['PG_PORT']
    database = os.environ['PG_DATABASE']
    user = os.environ['PG_USER']
    password = os.environ['PG_PASSWORD']

    # pd.read_sql用のコネクタ
    conn = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

    df_customer = pd.read_sql(sql='select * from customer', con=conn)
    df_category = pd.read_sql(sql='select * from category', con=conn)
    df_product = pd.read_sql(sql='select * from product', con=conn)
    df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
    df_store = pd.read_sql(sql='select * from store', con=conn)
    df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

else:
    if not os.path.exists('../data/'):
        !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
        os.chdir('100knocks-preprocess/docker/work/answer')

    dtype = {
        'customer_id': str,
        'gender_cd': str,
        'postal_cd': str,
        'application_store_cd': str,
        'status_cd': str,
        'category_major_cd': str,
        'category_medium_cd': str,
        'category_small_cd': str,
        'product_cd': str,
        'store_cd': str,
        'prefecture_cd': str,
        'tel_no': str,
        'postal_cd': str,
        'street': str
    }

    df_customer = pd.read_csv("../data/customer.csv", dtype=dtype)
    df_category = pd.read_csv("../data/category.csv", dtype=dtype)
    df_product = pd.read_csv("../data/product.csv", dtype=dtype)
    df_receipt = pd.read_csv("../data/receipt.csv", dtype=dtype)
    df_store = pd.read_csv("../data/store.csv", dtype=dtype)
    df_geocode = pd.read_csv("../data/geocode.csv", dtype=dtype)


Cloning into '100knocks-preprocess'...
remote: Enumerating objects: 1912, done.
remote: Counting objects: 100% (707/707), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 1912 (delta 589), reused 591 (delta 535), pack-reused 1205
Receiving objects: 100% (1912/1912), 20.97 MiB | 16.50 MiB/s, done.
Resolving deltas: 100% (1070/1070), done.


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df=df_receipt
df.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [ ]:
df[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [ ]:
df = df.rename(columns = {'sales_ymd':'sales_date'})
print(df[['sales_date', 'customer_id', 'product_cd', 'amount']].head(10))
df = df.rename(columns = {'sales_date':'sales_ymd'})

   sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680


---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df.loc[df.loc[:, 'customer_id']== 'CS018205000001']

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
36,20180911,1536624000,S13018,1122,2,CS018205000001,P071401012,1,2200
9843,20180414,1523664000,S13018,1142,2,CS018205000001,P060104007,6,600
21110,20170614,1497398400,S13018,1112,2,CS018205000001,P050206001,5,990
27673,20170614,1497398400,S13018,1112,1,CS018205000001,P060702015,1,108
27840,20190216,1550275200,S13018,1152,2,CS018205000001,P071005024,1,102
28757,20180414,1523664000,S13018,1142,1,CS018205000001,P071101002,1,278
39256,20190226,1551139200,S13018,1132,2,CS018205000001,P070902035,1,168
58121,20190924,1569283200,S13018,1102,1,CS018205000001,P060805001,1,495
68117,20190226,1551139200,S13018,1132,1,CS018205000001,P071401020,1,2200
72254,20180911,1536624000,S13018,1122,1,CS018205000001,P071401005,1,1100


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_select = df[['sales_ymd', 'customer_id', 'product_cd','amount']]
df2 = df_select.loc[(df_select['customer_id'] == 'CS018205000001') & (df_select['amount'] >= 1000)]
df2

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


**論理演算子について**  
| は「または」を意味し、~ は「否定」を意味します。これらの演算子も条件を組み合わせる際に使用できる。~は否定演算子であり、論理演算子の出力を逆点させる。  
**loc関数**  
loc関数はラベルで直接指定することもできるが、条件式で指定することもできる。&を使うときは各条件式を()でくくる。

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_select = df[['sales_ymd', 'customer_id','product_cd', 'quantity', 'amount']]
df2 = df_select.loc[(df_select['customer_id'] == "CS018205000001") & (df_select['amount'] >=1000) | (df_select['quantity']   >= 5)]
df2

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
47082,20190610,CS037414000080,P090401002,8,640
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
#後から抽出の方が楽
df2 = df.loc[(df['customer_id'] == 'CS018205000001') & (df['amount'] >=1000) & (df['amount'] <=2000)]
df_select = df2[['sales_ymd', 'customer_id','product_cd', 'amount']]
df_select

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df = df_receipt
df2 = df.loc[(df['customer_id'] == "CS018205000001") & ~(df['product_cd'] == "P071401019")]
df_select = df2[['sales_ymd', 'customer_id','product_cd', 'amount']]
df_select

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
>
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


In [ ]:
df_store.query('(~(prefecture_cd == "13") & floor_area < 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [ ]:
df = df_store
df2 = df.loc[df['store_cd'].str.contains('S14')]
#.strを入れることでstrに対する操作が可能
df2.head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


**シリーズの属性**  
series.dtypes - シリーズの値の型を返す  
series.shape - シリーズの次元数とサイズを返すタプル  
series.size - シリーズの要素数を返す  
series.ndim - シリーズの次元数(常に1)を返す  
series.values - シリーズの実際の値を含むNumPy配列を返す  
series.index - シリーズのインデックスラベルを返す  

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [ ]:
df = df_customer
#文字列の末尾を参照しブール値を返す、loc関数に渡す。
df2 = df.loc[df['customer_id'].str.endswith('1')]
df2.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [ ]:
df = df_store
df.loc[df['address'].str.contains('横浜')]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [ ]:
df = df_customer
patternes = ('A','B','C','D','E','F')
#startswith関数にはタプルを渡すこともできる
df2 = df.loc[df['status_cd'].str.startswith(patternes)]
df2.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
df = df_customer
pat = ('1','2','3','4','5','6','7','8','9')
df2 =df.loc[df['status_cd'].str.endswith(pat)]
df2.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
patternes = ('A','B','C','D','E','F')
pat = ('1','2','3','4','5','6','7','8','9')
df2 = df.loc[(df['status_cd'].str.startswith(patternes)) & (df['status_cd'].str.endswith(pat))]
df2.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df = df_store
#splitで-を境に分離
Ob = df['tel_no'].str.split('-')
print(Ob.head())
#Obから各リストにアクセスして要素を取り出し、長さを空のリストに格納する。
X = []
for i in Ob:
  for j in i:
    X.append(len(j))
print(X)
#データフレームに変換
num = pd.DataFrame(X)
#ナンパイ配列に変換してリシャープメソッドで整形
num_data = num.values.reshape(-1,3)
#データフレームに戻してラベリング
df_num =pd.DataFrame(num_data, columns=['first','second','third'])
#コンカット関数で結合
df2 = pd.concat([df, df_num], axis=1)
#loc関数で指定
A = df2.loc[(df2['first'] == 3) & (df2['second']== 3) & (df2['third'] == 4)]
A.head()

0    [043, 123, 4003]
1    [042, 123, 4008]
2    [045, 123, 4032]
3    [045, 123, 4043]
4    [042, 123, 4045]
Name: tel_no, dtype: object
[3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 2, 4, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 2, 4, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 4, 3, 3, 4]


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area,first,second,third
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0,3,3,4
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0,3,3,4
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0,3,3,4
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0,3,3,4
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0,3,3,4


---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
df = df_customer
df2 = df.sort_values(by='birth_day',ascending = True)
df2.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
df.sort_values(by='birth_day', ascending=True)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
...,...,...,...,...,...,...,...,...,...,...,...
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0


---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']]
                     ,df_receipt['amount'].rank(method='min',
                                                ascending=False)], axis=1)

df_tmp.columns = ['customer_id', 'amount', 'ranking']

df_tmp.sort_values('ranking').head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df = df_receipt
#amount順にソート降順
df2 = df.sort_values(by='amount',ascending = False)
# ランクを初期化
rank = 1
rank_list = []
# データフレームの行をループ
for i in range(len(df2)-1):
    x = df.iloc[i, 8]
    y = df.iloc[i + 1, 8]
    rank_list.append(rank)
    rank += 1
rank_list.append(rank_list[-1])

In [ ]:
#データフレームに変換して結合,インデックスの初期化
rank_data=pd.DataFrame(rank_list, columns = ['rank']).reset_index(drop = True)
df2.reset_index(drop = True, inplace = True)
df3 = pd.concat([df2[['customer_id','amount']],rank_data], axis=1)
df3.head(10)

,customer_id,amount,rank
0,CS011415000006,10925,1
1,ZZ000000000000,6800,2
2,CS028605000002,5780,3
3,ZZ000000000000,5480,4
4,CS015515000034,5480,5
5,ZZ000000000000,5480,6
6,ZZ000000000000,5440,7
7,CS021515000089,5440,8
8,CS008513000063,5280,9
9,CS032414000072,5280,10


---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [ ]:
df_receipt.count()

sales_ymd         104681
sales_epoch       104681
store_cd          104681
receipt_no        104681
receipt_sub_no    104681
customer_id       104681
product_cd        104681
quantity          104681
amount            104681
dtype: int64

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
df = df_receipt
df['customer_id'].nunique()

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df2 = df[['store_cd', 'amount', 'quantity']].groupby('store_cd').count().unstack()
df2

          store_cd
amount    S12007      2076
          S12013      2385
          S12014      2333
          S12029      2518
          S12030      2372
                      ... 
quantity  S14046      1335
          S14047      1025
          S14048       760
          S14049       775
          S14050       575
Length: 104, dtype: int64

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df2 = df[['customer_id','sales_ymd']].sort_values(by='sales_ymd', ascending=True)
#顧客id毎に集計して、アプライメソッドを適用しリストに格納。
grouped_df2 = df2.groupby('customer_id')['sales_ymd'].apply(list)
#各リストの最初の要素を取り出す
new = grouped_df2.apply(lambda x: x[0])
new.head(10)

customer_id
CS001113000004    20190308
CS001114000005    20180503
CS001115000010    20171228
CS001205000004    20170914
CS001205000006    20180207
CS001211000025    20190322
CS001212000027    20170127
CS001212000031    20180906
CS001212000046    20170811
CS001212000070    20191018
Name: sales_ymd, dtype: int64

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
#ascendingをFalseで指定
df2 = df[['customer_id','sales_ymd']].sort_values(by='sales_ymd', ascending=False)
#顧客id毎に集計して、アプライメソッドを適用しリストに格納。10件表示
grouped_df2 = df2.groupby('customer_id')['sales_ymd'].apply(list)
#最初の列に格納された日付を取り出す
old = grouped_df2.apply(lambda x:x[0])
old.head(10)

customer_id
CS001113000004    20190308
CS001114000005    20190731
CS001115000010    20190405
CS001205000004    20190625
CS001205000006    20190224
CS001211000025    20190322
CS001212000027    20170127
CS001212000031    20180906
CS001212000046    20170811
CS001212000070    20191018
Name: sales_ymd, dtype: int64

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
df_new = df[['customer_id','sales_ymd']].sort_values(by='sales_ymd', ascending=True)
#顧客id毎に集計して、アプライメソッドを適用しリストに格納。
grouped_df_new = df_new.groupby('customer_id')['sales_ymd'].apply(list)
grouped_df_new = grouped_df_new.reset_index()
#ascendingをFalseで指定
df_old = df[['customer_id','sales_ymd']].sort_values(by='sales_ymd', ascending=False)
#顧客id毎に集計して、アプライメソッドを適用しリストに格納。
grouped_df_old = df_old.groupby('customer_id')['sales_ymd'].apply(list)
grouped_df_old = grouped_df_old.reset_index()
#loc関数に渡す条件式を作成
condition = grouped_df_old['sales_ymd'] == grouped_df_new['sales_ymd']
#~否定演算子で条件式を否定
selected_rows = grouped_df_new.loc[~condition].reset_index(drop=True)
selected_rows.head(10)

,customer_id,sales_ymd
0,CS001114000005,"[20180503, 20180503, 20190731, 20190731]"
1,CS001115000010,"[20171228, 20171228, 20180701, 20180701, 20190..."
2,CS001205000004,"[20170914, 20170914, 20180821, 20180821, 20180..."
3,CS001205000006,"[20180207, 20180207, 20180415, 20180415, 20180..."
4,CS001214000009,"[20170306, 20170306, 20170601, 20170601, 20170..."
5,CS001214000017,"[20180828, 20180828, 20180905, 20180905, 20181..."
6,CS001214000048,"[20171109, 20171109, 20180617, 20180617, 20190..."
7,CS001214000052,"[20180208, 20180208, 20180331, 20180331, 20190..."
8,CS001215000005,"[20170206, 20170206, 20180514, 20180514, 20181..."
9,CS001215000040,"[20170214, 20170214, 20170219, 20170219, 20170..."


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df = df_receipt
df2 = df[['store_cd', 'amount']].groupby('store_cd').mean().sort_values(by='amount',
      ascending = False)
df2.head()

,amount
store_cd,
S13052,402.867470
S13015,351.111960
S13003,350.915519
S14010,348.791262
S13001,348.470386


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df = df_receipt
df2 = df[['store_cd', 'amount']].groupby('store_cd').median().sort_values(by='amount',
      ascending = False)
df2.head()

,amount
store_cd,
S13052,190.0
S14010,188.0
S14050,185.0
S14040,180.0
S13003,180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [ ]:
df = df_receipt
df_mode = df.groupby('store_cd')['product_cd'].apply(lambda x: x.value_counts().index[0])
df_mode.head(10)

store_cd
S12007    P060303001
S12013    P060303001
S12014    P060303001
S12029    P060303001
S12030    P060303001
S13001    P060303001
S13002    P060303001
S13003    P071401001
S13004    P060303001
S13005    P040503001
Name: product_cd, dtype: object

lamda x:でproduct_cdをxに指定し、value_countがグルーピングされた各product_cdに対して出現回数を返し、それが出現回数順にソートされているので、index[0]で最初の物を取り出せば最頻値を取り出すことができる

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [ ]:
df = df_receipt
df2 = df[['store_cd', 'amount']].groupby('store_cd').var().sort_values(by='amount',
                                                                       ascending=False)
df2.head(5)

,amount
store_cd,
S13052,441863.252526
S14011,306442.242432
S14034,297068.392740
S13001,295558.842618
S13015,295427.197086


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [ ]:
df = df_receipt
df2 = df[['store_cd', 'amount']].groupby('store_cd').std().sort_values(by='amount', ascending =False)
df2.head()

,amount
store_cd,
S13052,664.727954
S14011,553.572256
S14034,545.039808
S13001,543.653237
S13015,543.532149


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
import numpy as np
df = df_receipt['amount']
df.values
div = (25, 50, 75, 100)
percentiles =np.percentile(df,div)
print(percentiles)

[  102.   170.   288. 10925.]


---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df = df_receipt[['store_cd', 'amount']].groupby('store_cd').mean()
df.loc[df['amount'] >= 330]

,amount
store_cd,
S12013,330.194130
S13001,348.470386
S13003,350.915519
S13004,330.943949
S13015,351.111960
S13019,330.208616
S13020,337.879932
S13052,402.867470
S14010,348.791262


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
df = df_receipt[['customer_id','amount']]
init = df.loc[~df['customer_id'].str.startswith('Z')].groupby('customer_id').mean()
init

,amount
customer_id,
CS001113000004,649.000000
CS001114000005,156.500000
CS001115000010,507.333333
CS001205000004,198.800000
CS001205000006,278.083333
...,...
CS051212000001,168.000000
CS051513000004,275.500000
CS051515000002,132.500000


---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
df = df_receipt[['customer_id','amount']]
init = df.loc[~df['customer_id'].str.startswith('Z')].groupby('customer_id').amount.sum()
cus = init.loc[init>= init.mean()].reset_index()
cus

,customer_id,amount
0,CS001115000010,3044
1,CS001205000006,3337
2,CS001214000009,4685
3,CS001214000017,4132
4,CS001214000052,5639
...,...,...
2991,CS048415000017,4484
2992,CS048415000020,2995
2993,CS048513000001,2570
2994,CS049513000008,4450


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [ ]:
df = pd.merge(df_receipt, df_store[['store_cd', 'store_name']], how='inner', on ='store_cd')
df.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1542326400,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1484697600,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1558656000,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1555632000,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店
5,20181119,1542585600,S14006,1152,2,ZZ000000000000,P050701001,1,88,葛が谷店
6,20171211,1512950400,S14006,1132,2,CS006515000175,P090903001,1,80,葛が谷店
7,20191021,1571616000,S14006,1112,2,CS006415000221,P040602001,1,405,葛が谷店
8,20170710,1499644800,S14006,1132,2,CS006411000036,P090301051,1,330,葛が谷店
9,20190805,1564963200,S14006,112,1,CS006211000012,P050104001,1,115,葛が谷店


---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [ ]:
df = pd.merge(df_product, df_category[['category_small_cd', 'category_small_name']], how='inner',
              on='category_small_cd')
df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類
5,P040101006,04,0401,040101,298.0,224.0,弁当類
6,P040101007,04,0401,040101,338.0,254.0,弁当類
7,P040101008,04,0401,040101,420.0,315.0,弁当類
8,P040101009,04,0401,040101,498.0,374.0,弁当類
9,P040101010,04,0401,040101,580.0,435.0,弁当類


---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
df = df_receipt.groupby('customer_id').amount.sum().reset_index()
#reset_indexでcustomer_idをインデックスから列に変換、groupbyメソッドでインデックス化されているため、merge()で指定できなくなるから。
df1 = df_customer.loc[(df_customer['gender_cd']== '1') & ~(df_customer['customer_id'].str.startswith('Z'))]
merged =pd.merge(df1['customer_id'], df, how='left', on='customer_id').fillna(0)
#左結合では、左側の引数の結果が保持され、対応するキー（onで指定)がある場所は右の値を挿入している。
merged.head(10)
#引数の左は項目を作成しているだけで、値は右を参照していて、右の値がなければNaが返ってくるため、欠損地処理をしている。

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
#非会員を除外
df = df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
#売上日の重複を排除し、売上日数の多い顧客上位20件を
df_selected = df.loc[~df.duplicated(df[['customer_id', 'sales_ymd']])]
df1 = df_selected.groupby('customer_id').sales_ymd.count().reset_index().sort_values('sales_ymd', ascending=False).head(20)
df2 = df.groupby('customer_id').amount.sum().reset_index().sort_values('amount', ascending=False).head(20)
#完全外部結合を実装
pd.merge(df1,df2, how= 'outer', on ='customer_id')

,customer_id,sales_ymd,amount
0,CS040214000008,23.0,NaN
1,CS015415000185,22.0,20153.0
2,CS010214000010,22.0,18585.0
3,CS010214000002,21.0,NaN
4,CS028415000007,21.0,19127.0
5,CS017415000097,20.0,23086.0
6,CS016415000141,20.0,18372.0
7,CS031414000051,19.0,19202.0
8,CS014214000023,19.0,NaN
9,CS022515000226,19.0,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [ ]:
df1 = df_store.copy()
df2 = df_product.copy()
df1['key'] = 0
df2['key'] = 0
merged = pd.merge(df1, df2, how = 'outer', on ='key')
len(merged)

NameError: name 'df_store' is not defined

In [ ]:
result = pd.concat((df1, df2), axis=1)
result.drop('key', axis = 1, inplace = True)
result.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0,P040101001,04,0401,040101,198.0,149.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0,P040101002,04,0401,040101,218.0,164.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0,P040101003,04,0401,040101,230.0,173.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0,P040101004,04,0401,040101,248.0,186.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0,P040101005,04,0401,040101,268.0,201.0


In [ ]:
merged.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area,key,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0,0,P040101001,04,0401,040101,198.0,149.0
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0,0,P040101002,04,0401,040101,218.0,164.0
2,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0,0,P040101003,04,0401,040101,230.0,173.0
3,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0,0,P040101004,04,0401,040101,248.0,186.0
4,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0,0,P040101005,04,0401,040101,268.0,201.0


---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [ ]:
df = df_receipt.groupby('sales_ymd').amount.sum()
diff = df.diff().dropna()
diff.head(10)

sales_ymd
20170102   -9558.0
20170103    3338.0
20170104    8662.0
20170105    1665.0
20170106   -5443.0
20170107   -8972.0
20170108    1322.0
20170109    1981.0
20170110   -6575.0
20170111    4144.0
Name: amount, dtype: float64

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [ ]:
df = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_lag = df
for i  in range(1, 4):
  #シフトしたdfを結合
  #df_lagが定義され続けるため、shiftでずらすのは定義が変わらないdf
  df_lag = pd.concat((df_lag, df.shift(i)), axis=1)
  #新たなカラムズを定義
  columns = [f'shift{i}_ymd', f'shift{i}_amount']
  #重複したカラムズを消去して作成したカラムズを追加
  df_lag.columns = list(df_lag.columns)[:-len(columns)] + columns
df_lag.dropna().astype(int).sort_values(['sales_ymd'], ascending= True).head(10)

,sales_ymd,amount,shift1_ymd,shift1_amount,shift2_ymd,shift2_amount,shift3_ymd,shift3_amount
3,20170104,36165,20170103,27503,20170102,24165,20170101,33723
4,20170105,37830,20170104,36165,20170103,27503,20170102,24165
5,20170106,32387,20170105,37830,20170104,36165,20170103,27503
6,20170107,23415,20170106,32387,20170105,37830,20170104,36165
7,20170108,24737,20170107,23415,20170106,32387,20170105,37830
8,20170109,26718,20170108,24737,20170107,23415,20170106,32387
9,20170110,20143,20170109,26718,20170108,24737,20170107,23415
10,20170111,24287,20170110,20143,20170109,26718,20170108,24737
11,20170112,23526,20170111,24287,20170110,20143,20170109,26718
12,20170113,28004,20170112,23526,20170111,24287,20170110,20143


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
#結合
merged = pd.merge(df_receipt, df_customer, how= 'inner', on='customer_id')
#グルーピング
df = merged.groupby(['gender_cd', 'age']).amount.sum().reset_index()
#cut関数でグループを作成
df['age_group'] = pd.cut(df['age'], bins=range(0, 101, 10), right =False, labels =False)*10
cro = df.groupby(['age_group', 'gender_cd']).amount.sum().unstack()
cro.columns = ['男性', '女性', '不明']
cro

,男性,女性,不明
age_group,,,
10,1591.0,149836.0,4317.0
20,72940.0,1363724.0,44328.0
30,177322.0,693047.0,50441.0
40,19355.0,9320791.0,483512.0
50,54320.0,6685192.0,342923.0
60,272469.0,987741.0,71418.0
70,13435.0,29764.0,2427.0
80,46360.0,262923.0,5111.0
90,NaN,6260.0,NaN


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [ ]:
cro['gender']=cro['gender_cd'].astype(int).map({0: '男性',1: '女性', 9:'不明'})
cro['new_gender_cd'] = cro['gender_cd'].astype(int).map({0: '00', 1:'01', 9:'99'})
cro.drop('gender_cd', axis=1, inplace=True)
cro.set_index('new_gender_cd')

In [ ]:
#結合
merged = pd.merge(df_receipt, df_customer, how= 'inner', on='customer_id')
#グルーピング
df = merged.groupby(['gender_cd', 'age']).amount.sum().reset_index()
#cut関数でグループを作成
df['age_group'] = pd.cut(df['age'], bins=range(0, 101, 10), right =False, labels =False)*10
df['gender_cd'] = df['gender_cd'].astype(int).map({0: '00', 1:'01', 9:'99'})
cro = df.groupby(['age_group','gender_cd']).amount.sum().reset_index()
cro

,age_group,gender_cd,amount
0,10,00,1591
1,10,01,149836
2,10,99,4317
3,20,00,72940
4,20,01,1363724
5,20,99,44328
6,30,00,177322
7,30,01,693047
8,30,99,50441
9,40,00,19355


---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
df = pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')],axis=1)
df.head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['application_date'], format='%Y%m%d')], axis=1).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
df = df_receipt
df1 = pd.concat([df[['receipt_no','receipt_sub_no']],
                 pd.to_datetime(df['sales_ymd'].astype('str'), format='%Y%m%d')], axis=1)
df1.head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
df = df_receipt
#エポック秒を日付型に変換した後、文字列型に変換してyearを取り出す
years = pd.to_datetime(df['sales_epoch'], unit='s').rename('sales_ymd')
#unixエポック秒を日付型に変換するための引数：unit=s
pd.concat([df[['receipt_no', 'receipt_sub_no']], years], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
#数値型で取得したものを.dtメソッドで文字列型に変換した後、strftimeでフォーマットを指定
data = pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%Y')
#結合
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']], data], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [ ]:
df = df_receipt
#数値型として取得し、文字型に変換し、フォーマットを指定して月だけを取り出す
data = pd.to_datetime(df['sales_epoch'], unit = 's').dt.strftime('%m').rename('sales_m')
pd.concat([df[['receipt_no', 'receipt_sub_no']], data], axis=1).head(10)

,receipt_no,receipt_sub_no,sales_m
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [ ]:
df = df_receipt
#数字型で取り出す、.dtメソッドで日付型に変換、strftimeでフォーマット指定
date  =pd.to_datetime(df['sales_epoch'], unit='s').dt.strftime('%d').rename('sales_date')
pd.concat([df[['receipt_no', 'receipt_sub_no']], date], axis=1)

,receipt_no,receipt_sub_no,sales_date
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
...,...,...,...
104676,1132,2,21
104677,1132,2,11
104678,1122,1,11
104679,1142,1,31


---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
#顧客idzから始まるものを排除
df= df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
#顧客id毎に集計
grouped = df.groupby('customer_id').amount.sum()
#groupedの値に対してブール値を判定
judge = grouped.apply(lambda x: 0 if x <= 2000 else 1).rename('judge')
new_df = pd.concat([grouped,judge], axis=1)
new_df.head(10)

,amount,judge
customer_id,,
CS001113000004,1298,0
CS001114000005,626,0
CS001115000010,3044,1
CS001205000004,1988,0
CS001205000006,3337,1
CS001211000025,456,0
CS001212000027,448,0
CS001212000031,296,0
CS001212000046,228,0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [ ]:
df = df_customer

In [ ]:
#郵便番号を取り出し、データフレームへ結合
lists = []
code = df['postal_cd'].str.split('-')
for i in code:
  for j in i:
    lists.append(j)
df_code = pd.DataFrame(lists).values
df_code = df_code.reshape(-1, 2)
df_code = pd.DataFrame(df_code, columns=['first', 'second'])
new_df = pd.concat([df, df_code], axis=1)

In [ ]:
#無名関数で条件式を適合しブール値を作成して結合
judge = new_df['first'].astype(int).apply(lambda x : 1 if 100 <= x <=209 else 0).rename('judge')
new_df = pd.concat([new_df, judge], axis=1)
new_df.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,first,second,judge
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,259,1113,0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,136,0076,1
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,151,0053,1
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,245,0016,0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,144,0055,1


In [ ]:
#df_receiptとマージし集計
merged = pd.merge(new_df, df_receipt, how = 'inner', on = 'customer_id')
grouped = merged.groupby('judge').customer_id.agg({'nunique'})
grouped

,nunique
judge,
0,3906
1,4400


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [ ]:
df = df_customer
df['addressing'] = df['address'].str[0:3].map({'埼玉県':11,  '千葉県':12, '東京都':13, '神奈川':14})
df[['customer_id', 'address', 'addressing']].head(10)

,customer_id,address,addressing
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [ ]:
df = df_receipt.groupby('customer_id').amount.sum()
#四分位点に対応する値を取得
quantiles = np.arange(0.25,1,0.25)
Q = []
for i in quantiles:
  Q.append(df.quantile(i))
#関数を定義
def judged(x):
  if x < Q[0]:
    return 1
  elif Q[0]<= x < Q[1]:
    return 2
  elif Q[1]<= x < Q[2]:
    return 3
  else:
    return 4
#applyメソッドを用いて新しいカテゴリを作成、カラム名がamountで出力されるのでrename
new_cat = df.apply(judged).rename('judge')
new_df = pd.concat([df,new_cat], axis =1)
new_df.head(10)

,amount,judge
customer_id,,
CS001113000004,1298,2
CS001114000005,626,2
CS001115000010,3044,3
CS001205000004,1988,3
CS001205000006,3337,3
CS001211000025,456,1
CS001212000027,448,1
CS001212000031,296,1
CS001212000046,228,1


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [ ]:
df = df_customer
#60代までをcut関数を用いてフィルタリング、NA値を60で補填
#cut関数はビンの幅を指定し、該当する階級を出力する
age_group = pd.cut(df['age'], bins = np.arange(0, 70, 10), right =False, labels=False)*10
age_group = age_group.fillna(60)
new_df = pd.concat([df[['customer_id', 'birth_day']], age_group], axis =1)
new_df.head(10)

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,30.0
1,CS037613000071,1952-04-01,60.0
2,CS031415000172,1976-10-04,40.0
3,CS028811000001,1933-03-27,60.0
4,CS001215000145,1995-03-29,20.0
5,CS020401000016,1974-09-15,40.0
6,CS015414000103,1977-08-09,40.0
7,CS029403000008,1973-08-17,40.0
8,CS015804000004,1931-05-02,60.0
9,CS033513000180,1962-07-11,50.0


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [ ]:
df = df_customer
#60代までをcut関数を用いてフィルタリング、NA値を60で補填
#cut関数はビンの幅を指定し、該当する階級を出力する
age_group = pd.cut(df['age'], bins=np.arange(0,70,10), right=False, labels=False)*10
age_group = age_group.fillna(60)
#新たな列を作成
sexial_age = df['gender_cd'].astype('str') + age_group.astype('int').astype('str')
#カラム名を変更
new_df = pd.concat([df[['customer_id', 'birth_day']], age_group, sexial_age], axis =1)
new_df.head(10)

,customer_id,birth_day,age,0
0,CS021313000114,1981-04-29,30.0,130
1,CS037613000071,1952-04-01,60.0,960
2,CS031415000172,1976-10-04,40.0,140
3,CS028811000001,1933-03-27,60.0,160
4,CS001215000145,1995-03-29,20.0,120
5,CS020401000016,1974-09-15,40.0,040
6,CS015414000103,1977-08-09,40.0,140
7,CS029403000008,1973-08-17,40.0,040
8,CS015804000004,1931-05-02,60.0,060
9,CS033513000180,1962-07-11,50.0,150


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
df = df_customer
dum = pd.get_dummies(df['gender_cd'])
C = ['男性','女性', '不明']
dum.columns = C
new_df = pd.concat([df['customer_id'], dum], axis=1)
new_df.head(10)

,customer_id,男性,女性,不明
0,CS021313000114,False,True,False
1,CS037613000071,False,False,True
2,CS031415000172,False,True,False
3,CS028811000001,False,True,False
4,CS001215000145,False,True,False
5,CS020401000016,True,False,False
6,CS015414000103,False,True,False
7,CS029403000008,True,False,False
8,CS015804000004,True,False,False
9,CS033513000180,False,True,False


---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


In [ ]:
#関数を呼び出す
from sklearn.preprocessing  import StandardScaler
#非会員を除外
df = df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
#集計してnumpy配列に変換
grouped = df.groupby('customer_id').amount.sum().values
#二次元データに変換
grouped2 = grouped.reshape(-1, 1)
#インスタンス化
scaler = StandardScaler()
data_fit = scaler.fit_transform(grouped2)
#データフレームに変換
grouped_df = df.groupby('customer_id').amount.sum().reset_index()
new_df = pd.DataFrame(data_fit, columns=['standard'])
new_df2 =pd.concat([grouped_df, new_df], axis=1)
new_df2.head(10)

,customer_id,amount,standard
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df= df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
grouped = df.groupby('customer_id').amount.sum()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_fit = scaler.fit_transform(grouped.values.reshape(-1, 1))
grouped_df = df.groupby('customer_id').amount.sum()
fit_df = pd.DataFrame(data_fit).rename(columns = {0: scaler})
new_df = pd.concat([grouped_df.reset_index(), fit_df], axis=1)
new_df.head(10)

,customer_id,amount,MinMaxScaler()
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df = df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
grouped = df.groupby('customer_id').amount.sum()
Y = np.log10(grouped.values +0.5)
log10_df = pd.DataFrame(Y).rename(columns={0: 'log10'})
new_df = pd.concat([grouped.reset_index(),log10_df], axis=1)
new_df.head(10)

,customer_id,amount,log10
0,CS001113000004,1298,3.113442
1,CS001114000005,626,2.796921
2,CS001115000010,3044,3.483516
3,CS001205000004,1988,3.298526
4,CS001205000006,3337,3.523421
5,CS001211000025,456,2.659441
6,CS001212000027,448,2.651762
7,CS001212000031,296,2.472025
8,CS001212000046,228,2.358886
9,CS001212000070,456,2.659441


---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
#非会員を除外
df = df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
grouped = df.groupby('customer_id').amount.sum()
#自然対数化,配列化、dfに変換、結合
Y = np.log(grouped.values +0.5)
log_df = pd.DataFrame(Y).rename(columns={0: 'log'})
new_df = pd.concat([grouped.reset_index(), log_df], axis=1)
new_df.head(10)

,customer_id,amount,log
0,CS001113000004,1298,7.168965
1,CS001114000005,626,6.440149
2,CS001115000010,3044,8.021092
3,CS001205000004,1988,7.595136
4,CS001205000006,3337,8.112977
5,CS001211000025,456,6.123589
6,CS001212000027,448,6.105909
7,CS001212000031,296,5.692047
8,CS001212000046,228,5.431536
9,CS001212000070,456,6.123589


---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
df = df_product
df['profit'] = df['unit_price'] - df['unit_cost']
df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,profit
0,P040101001,04,0401,040101,198.0,149.0,49.0
1,P040101002,04,0401,040101,218.0,164.0,54.0
2,P040101003,04,0401,040101,230.0,173.0,57.0
3,P040101004,04,0401,040101,248.0,186.0,62.0
4,P040101005,04,0401,040101,268.0,201.0,67.0
5,P040101006,04,0401,040101,298.0,224.0,74.0
6,P040101007,04,0401,040101,338.0,254.0,84.0
7,P040101008,04,0401,040101,420.0,315.0,105.0
8,P040101009,04,0401,040101,498.0,374.0,124.0
9,P040101010,04,0401,040101,580.0,435.0,145.0


---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
#欠損地を確認
df.isnull().sum()
#欠損地を平均値で保管
df['unit_price'].fillna(df['unit_price'].mean())
df['unit_cost'].fillna(df['unit_cost'].mean())
#利益を格納
df['profit'] = df['unit_price'] - df['unit_cost']
#利益率を算出
df['profit_rate'] = df['profit']/df['unit_price']
df['profit_rate'].mean()

0.24911389885177

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
#欠損地を除外
df = df_product.copy()
df =df.loc[~df['unit_price'].isnull()]
df['new_price'] = np.floor(df['unit_cost'] / 0.7)
df['new_price_rate'] = (df['new_price'] - df['unit_cost'] )/ df['new_price']
df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_price_rate
0,P040101001,04,0401,040101,198.0,149.0,212.0,0.297170
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,265.0,0.298113
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,362.0,0.298343
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df = df_product[['product_cd','unit_price', 'unit_cost']].copy()
df['new_price'] =np.round(df['unit_cost'] / 0.7)
df['new_price_rate'] = (df['new_price'] - df['unit_cost']) / df['new_price']
df.head(10)

,product_cd,unit_price,unit_cost,new_price,new_price_rate
0,P040101001,198.0,149.0,213.0,0.300469
1,P040101002,218.0,164.0,234.0,0.299145
2,P040101003,230.0,173.0,247.0,0.299595
3,P040101004,248.0,186.0,266.0,0.300752
4,P040101005,268.0,201.0,287.0,0.299652
5,P040101006,298.0,224.0,320.0,0.300000
6,P040101007,338.0,254.0,363.0,0.300275
7,P040101008,420.0,315.0,450.0,0.300000
8,P040101009,498.0,374.0,534.0,0.299625
9,P040101010,580.0,435.0,621.0,0.299517


---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
df = df_product[['product_cd','unit_price', 'unit_cost']].copy()
df['new_price'] =np.ceil(df['unit_cost'] / 0.7)
df['new_price_rate'] = (df['new_price'] - df['unit_cost']) / df['new_price']
df.head(10)

,product_cd,unit_price,unit_cost,new_price,new_price_rate
0,P040101001,198.0,149.0,213.0,0.300469
1,P040101002,218.0,164.0,235.0,0.302128
2,P040101003,230.0,173.0,248.0,0.302419
3,P040101004,248.0,186.0,266.0,0.300752
4,P040101005,268.0,201.0,288.0,0.302083
5,P040101006,298.0,224.0,320.0,0.300000
6,P040101007,338.0,254.0,363.0,0.300275
7,P040101008,420.0,315.0,451.0,0.301552
8,P040101009,498.0,374.0,535.0,0.300935
9,P040101010,580.0,435.0,622.0,0.300643


---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
df = df_product[['product_cd','unit_price', 'unit_cost']].copy()
#欠損値を除外
df = df.loc[~df['unit_price'].isnull()]
df['tax_price'] = df['unit_price']*0.1
df['tax_unit_price'] = np.floor(df['unit_price'] + df['tax_price'])
df.head(10)

,product_cd,unit_price,unit_cost,tax_price,tax_unit_price
0,P040101001,198.0,149.0,19.8,217.0
1,P040101002,218.0,164.0,21.8,239.0
2,P040101003,230.0,173.0,23.0,253.0
3,P040101004,248.0,186.0,24.8,272.0
4,P040101005,268.0,201.0,26.8,294.0
5,P040101006,298.0,224.0,29.8,327.0
6,P040101007,338.0,254.0,33.8,371.0
7,P040101008,420.0,315.0,42.0,462.0
8,P040101009,498.0,374.0,49.8,547.0
9,P040101010,580.0,435.0,58.0,638.0


---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
#結合
df = pd.merge(df_receipt, df_product, how='inner', on = 'product_cd')
#集計
grouped = df.groupby(['customer_id', 'category_major_cd']).amount.sum().unstack()
#選択
selected = grouped.loc[~grouped['07'].isnull()]
#各列のリストを取得
L = selected.columns.tolist()
#各列の総和を計算
selected['sum'] = selected[L].sum(axis=1)
selected['rate'] = selected['07'] / selected['sum']
selected.head(10)

<ipython-input-70-988ee164d9c1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['sum'] = selected[L].sum(axis=1)
<ipython-input-70-988ee164d9c1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['rate'] = selected['07'] / selected['sum']


category_major_cd,04,05,06,07,08,09,sum,rate
customer_id,,,,,,,,
CS001113000004,NaN,NaN,NaN,1298.0,NaN,NaN,1298.0,1.000000
CS001114000005,NaN,40.0,NaN,486.0,100.0,NaN,626.0,0.776358
CS001115000010,NaN,NaN,NaN,2694.0,NaN,350.0,3044.0,0.885020
CS001205000004,100.0,128.0,286.0,346.0,368.0,760.0,1988.0,0.174044
CS001205000006,635.0,60.0,198.0,2004.0,80.0,360.0,3337.0,0.600539
CS001212000027,248.0,NaN,NaN,200.0,NaN,NaN,448.0,0.446429
CS001212000031,NaN,NaN,NaN,296.0,NaN,NaN,296.0,1.000000
CS001212000046,NaN,NaN,NaN,108.0,NaN,120.0,228.0,0.473684
CS001212000070,NaN,NaN,148.0,308.0,NaN,NaN,456.0,0.675439


---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
#重複を排除
df_tmp = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()
df = pd.merge(df_tmp, df_customer, how='inner', on = 'customer_id')
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'].astype('str'))
df['application_date'] = pd.to_datetime(df['application_date'].astype('str'))
df['diff'] = df['sales_ymd'] - df['application_date']
df['diff'] = df['diff'].dt.days
df.head(10)

,customer_id,sales_ymd,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,diff
0,CS006214000001,2018-11-03,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1371
1,CS006214000001,2017-05-09,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,828
2,CS006214000001,2017-06-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,858
3,CS006214000001,2018-10-28,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1365
4,CS006214000001,2019-09-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1680
5,CS006214000001,2018-01-31,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1095
6,CS006214000001,2017-07-05,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,885
7,CS006214000001,2018-11-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1378
8,CS006214000001,2019-04-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1529
9,CS006214000001,2019-06-01,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,1581


---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()
df = pd.merge(df_tmp, df_customer, how='inner', on = 'customer_id')
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'].astype('str'))
df['application_date'] = pd.to_datetime(df['application_date'].astype('str'))
df['diff'] = df['sales_ymd'] - df['application_date']
#一か月は30.44日で計算されるのが一般的
df['diff'] = np.floor(df['diff'] / pd.Timedelta(days=30.44))
df.head(10)

,customer_id,sales_ymd,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,diff
0,CS006214000001,2018-11-03,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,45.0
1,CS006214000001,2017-05-09,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,27.0
2,CS006214000001,2017-06-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,28.0
3,CS006214000001,2018-10-28,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,44.0
4,CS006214000001,2019-09-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,55.0
5,CS006214000001,2018-01-31,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,35.0
6,CS006214000001,2017-07-05,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,29.0
7,CS006214000001,2018-11-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,45.0
8,CS006214000001,2019-04-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,50.0
9,CS006214000001,2019-06-01,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,51.0


---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()
df = pd.merge(df_tmp, df_customer, how='inner', on = 'customer_id')
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'].astype('str'))
df['application_date'] = pd.to_datetime(df['application_date'].astype('str'))
df['diff'] = df['sales_ymd'] - df['application_date']
#一か月は30.44日で計算されるのが一般的
df['diff'] = np.floor(df['diff'] / pd.Timedelta(days=365))
df.head(10)

,customer_id,sales_ymd,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,diff
0,CS006214000001,2018-11-03,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,3.0
1,CS006214000001,2017-05-09,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,2.0
2,CS006214000001,2017-06-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,2.0
3,CS006214000001,2018-10-28,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,3.0
4,CS006214000001,2019-09-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,4.0
5,CS006214000001,2018-01-31,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,3.0
6,CS006214000001,2017-07-05,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,2.0
7,CS006214000001,2018-11-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,3.0
8,CS006214000001,2019-04-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,4.0
9,CS006214000001,2019-06-01,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,4.0


---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
df_tmp = df_receipt[['customer_id', 'sales_ymd']].drop_duplicates()
df = pd.merge(df_tmp, df_customer, how='inner', on = 'customer_id')
df['sales_ymd'] = pd.to_datetime(df['sales_ymd'].astype('str'))
df['application_date'] = pd.to_datetime(df['application_date'].astype('str'))
#日付型データを整数値に変換して計算
df['diff'] = df['sales_ymd'].view(np.int64) - df['application_date'].view(np.int64)
df['diff'] = df['diff']/ 10**9
df.head(10)

,customer_id,sales_ymd,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,diff
0,CS006214000001,2018-11-03,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,118454400.0
1,CS006214000001,2017-05-09,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,71539200.0
2,CS006214000001,2017-06-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,74131200.0
3,CS006214000001,2018-10-28,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,117936000.0
4,CS006214000001,2019-09-08,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,145152000.0
5,CS006214000001,2018-01-31,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,94608000.0
6,CS006214000001,2017-07-05,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,76464000.0
7,CS006214000001,2018-11-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,119059200.0
8,CS006214000001,2019-04-10,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,132105600.0
9,CS006214000001,2019-06-01,志水 佳乃,1,女性,1996-12-08,22,224-0057,神奈川県横浜市都筑区川和町**********,S14006,2015-02-01,E-20100908-F,136598400.0


---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
#よくわからなかった・主にrerativedeltaの作用が何に対して、どのように動くのかがよくわからない
df_tmp = df_receipt[['sales_ymd']].copy()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))

df_tmp['elapsed_days']  = df_tmp['sales_ymd'].apply(lambda x:x.weekday())

df_tmp['monday'] = \
    df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))

df_tmp.head(10)

,sales_ymd,elapsed_days,monday
0,2018-11-03,5,2018-10-29
1,2018-11-18,6,2018-11-12
2,2017-07-12,2,2017-07-10
3,2019-02-05,1,2019-02-04
4,2018-08-21,1,2018-08-20
5,2019-06-05,2,2019-06-03
6,2018-12-05,2,2018-12-03
7,2019-09-22,6,2019-09-16
8,2017-05-04,3,2017-05-01
9,2019-10-10,3,2019-10-07


---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [ ]:
df_customer.sample(frac=0.01).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2540,CS003311000022,玉山 希,1,女性,1983-10-21,35,206-0811,東京都稲城市押立**********,S13003,20150723,0-00000000-0
16976,CS001613000168,山田 遥,1,女性,1955-07-03,63,144-0055,東京都大田区仲六郷**********,S13001,20160402,0-00000000-0
8038,CS014711000007,菅谷 夏希,1,女性,1944-04-08,74,264-0021,千葉県千葉市若葉区若松町**********,S12014,20150522,0-00000000-0
15621,CS001402000008,前島 育二,0,男性,1977-03-11,42,210-0001,神奈川県川崎市川崎区本町**********,S13001,20150707,0-00000000-0
2739,CS011614000002,大野 智花,1,女性,1951-03-15,68,223-0065,神奈川県横浜市港北区高田東**********,S14011,20150204,8-20100830-7
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
20335,CS011514000018,多田 遥,1,女性,1962-07-11,56,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20151111,A-20100731-A
6677,CS028513000061,本橋 奈月,1,女性,1963-04-02,55,241-0826,神奈川県横浜市旭区東希望が丘**********,S14028,20150718,2-20080114-5
4496,CS002514000083,ほしの 彩,1,女性,1965-03-25,54,185-0022,東京都国分寺市東元町**********,S13002,20150620,A-20100509-2
21067,CS019313000148,大矢 七世,1,女性,1984-10-27,34,176-0004,東京都練馬区小竹町**********,S13019,20150803,0-00000000-0


---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [ ]:
from sklearn.model_selection import train_test_split
_, df_tmp = train_test_split(df_customer, test_size= 0.1,
                             stratify= df_customer['gender_cd'])
df_tmp.groupby( 'gender_cd').customer_id.count()

gender_cd
0     298
1    1793
9     107
Name: customer_id, dtype: int64

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [ ]:
df = df_receipt
grouped = df.groupby('customer_id').amount.sum().reset_index()
grouped['log_amount'] = np.log(grouped['amount'] + 0.5)
grouped['amount_ss'] = preprocessing.scale(grouped['log_amount'])
result = grouped.loc[grouped['amount_ss'] > 3]
result.head()

,customer_id,amount,log_amount,amount_ss
8306,ZZ000000000000,12395003,16.332804,7.967822


---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [ ]:
df = df_receipt.loc[~df_receipt['customer_id'].str.startswith('Z')]
Q1 = np.percentile(grouped['amount'], q=25)
Q3 = np.percentile(grouped['amount'], q=75)
IQR = Q3 - Q1
grouped = df.groupby('customer_id').amount.sum().reset_index()
result = grouped.loc[(grouped['amount'] < Q1 - (1.5*IQR)) | (grouped['amount'] > Q3 + (1.5*IQR))]
result.head(10)

,customer_id,amount
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
print(df_product.shape)
df = df_product.copy().dropna()
df.shape

(10030, 6)


(10023, 6)

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df = df_product.copy()
df['unit_price'].fillna(np.round(df['unit_price'].mean()), inplace=True)
df['unit_cost'].fillna(np.round(df['unit_cost'].mean()), inplace=True)
df.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df = df_product.copy()
#中央値を格納
M_p = np.round(df['unit_price'].median())
M_c = np.round(df['unit_cost'].median())
df['unit_price'].fillna(M_p, inplace=True)
df['unit_cost'].fillna(M_c, inplace= True)
df.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df = df_product.copy()
#中央値を取得
grouped_p_m = df.groupby('category_small_cd').unit_price.median().reset_index()
grouped_c_m = df.groupby('category_small_cd').unit_cost.median().reset_index()
df['nam'] = df['unit_price'].isnull()
#インデックスを取得
I = df.loc[df['nam'] == True].index
#smallcdを取得し欠損地を埋める
for i in I:
  small_cd = df.loc[i, 'category_small_cd']
  fill_values_p = grouped_p_m.loc[grouped_p_m['category_small_cd'] == small_cd, 'unit_price'].values
  fill_values_c = grouped_c_m.loc[grouped_c_m['category_small_cd'] == small_cd, 'unit_cost'].values
  df.loc[i, 'unit_price'] = np.round(fill_values_p)
  df.loc[i, 'unit_cost'] = np.round(fill_values_c)
df.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
nam                   0
dtype: int64

In [ ]:
df.loc[I, 'unit_price']
df.loc[I, 'unit_cost']

159     235.0
196     100.0
496     134.0
1531    200.0
2012    179.0
6296    196.0
7075    521.0
Name: unit_cost, dtype: float64

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [ ]:
#結合
df = pd.merge(df_customer, df_receipt, how='inner', on = 'customer_id')
#2019であるかないかのブール値を新たな列に格納
df['2019'] = df['sales_ymd'].astype('str').str.startswith('2019')
g = df.groupby(['customer_id','2019']).amount.sum().unstack().fillna(0)
g['all'] = g[True]+g[False]
g['ratio'] = g[True] / (g[False]+g[True])
g=g.fillna(0)
print(g.loc[g['ratio']!=0].head(10))
g.isnull().sum()

2019             False    True     all     ratio
customer_id                                     
CS001113000004     0.0  1298.0  1298.0  1.000000
CS001114000005   438.0   188.0   626.0  0.300319
CS001115000010  2466.0   578.0  3044.0  0.189882
CS001205000004  1286.0   702.0  1988.0  0.353119
CS001205000006  2851.0   486.0  3337.0  0.145640
CS001211000025     0.0   456.0   456.0  1.000000
CS001212000070     0.0   456.0   456.0  1.000000
CS001214000009  4021.0   664.0  4685.0  0.141729
CS001214000017  1170.0  2962.0  4132.0  0.716844
CS001214000048   485.0  1889.0  2374.0  0.795703


2019
False    0
True     0
all      0
ratio    0
dtype: int64

In [ ]:
g.loc[I, 'ratio']

customer_id
CS031415000172    0.583923
CS015414000103    0.279949
CS011215000048    0.072009
CS029415000023    0.729050
CS035415000029    0.775986
CS023513000066    0.269780
CS035513000134    0.295847
CS001515000263    1.000000
CS006415000279    1.000000
CS031415000106    0.027774
Name: ratio, dtype: float64

In [ ]:
#解答との比較
df_receipt_2019 = df_receipt.query('20190101 <= sales_ymd <= 20191231') \
                            .groupby('customer_id') \
                            .agg(amount_2019=('amount', 'sum')) \
                            .reset_index()

df_receipt_all = df_receipt.groupby('customer_id')\
                            .agg(amount_all=('amount', 'sum')) \
                            .reset_index()

df_sales_rate = df_customer[['customer_id']] \
                    .merge(df_receipt_2019, how='left', on='customer_id') \
                    .merge(df_receipt_all, how='left', on='customer_id')

df_sales_rate['amount_2019'] = df_sales_rate['amount_2019'].fillna(0)
df_sales_rate['amount_all'] = df_sales_rate['amount_all'].fillna(0)

df_sales_rate['amount_rate'] = \
                df_sales_rate[['amount_2019','amount_all']] \
                    .apply(lambda x: 0 if x[0] == 0 else x[0] / x[1], axis=1)

df_sales_rate['amount_rate'] = df_sales_rate['amount_rate'].fillna(0)
#customer_id を取得
I = df_sales_rate.set_index('customer_id').query('amount_rate > 0').head(10).index

In [ ]:
#比較すべて同じ
print(g.loc[I, 'ratio'])
print(df_sales_rate.query('amount_rate > 0').head(10))

customer_id
CS031415000172    0.583923
CS015414000103    0.279949
CS011215000048    0.072009
CS029415000023    0.729050
CS035415000029    0.775986
CS023513000066    0.269780
CS035513000134    0.295847
CS001515000263    1.000000
CS006415000279    1.000000
CS031415000106    0.027774
Name: ratio, dtype: float64
       customer_id  amount_2019  amount_all  amount_rate
2   CS031415000172       2971.0      5088.0     0.583923
6   CS015414000103        874.0      3122.0     0.279949
12  CS011215000048        248.0      3444.0     0.072009
15  CS029415000023       3767.0      5167.0     0.729050
21  CS035415000029       5823.0      7504.0     0.775986
23  CS023513000066        208.0       771.0     0.269780
24  CS035513000134        463.0      1565.0     0.295847
27  CS001515000263        216.0       216.0     1.000000
30  CS006415000279        229.0       229.0     1.000000
32  CS031415000106        215.0      7741.0     0.027774


---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [ ]:
df= df_geocode.copy()
g_long = df.groupby('postal_cd').longitude.mean()
g_lati = df.groupby('postal_cd').latitude.mean()
g_long_lati = pd.concat([g_long, g_lati], axis=1)
result = pd.merge(df_customer, g_long_lati, how='inner', on = 'postal_cd')

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [ ]:
#lati:緯度 long:経度
result.rename(columns={'application_store_cd':'store_cd'}, inplace=True)
merged = pd.merge(result, df_store,  how='inner', on='store_cd')
def calc_distance_numpy(x1, y1, x2, y2):
    x1_r = np.radians(x1)
    x2_r = np.radians(x2)
    y1_r = np.radians(y1)
    y2_r = np.radians(y2)
    return 6371 * np.arccos(np.sin(x1_r) * np.sin(x2_r)
                            + np.cos(x1_r) * np.cos(x2_r)
                            * np.cos(y1_r - y2_r))
merged['distance'] = calc_distance_numpy(merged['latitude_x'],
                                         merged['longitude_x'],
                                         merged['latitude_y'],
                                         merged['longitude_y'])
merged.head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address_x,store_cd,application_date,...,store_name,prefecture_cd,prefecture,address_y,address_kana,tel_no,longitude_y,latitude_y,floor_area,distance
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
1,CS021303000023,堀 一徳,0,男性,1980-04-05,38,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160411,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
2,CS021303000007,石倉 俊二,0,男性,1987-07-04,31,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150707,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
3,CS021313000183,草野 未來,1,女性,1986-12-21,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20170611,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
4,CS021314000098,筒井 れいな,1,女性,1985-09-21,33,259-1113,神奈川県伊勢原市粟窪**********,S14021,20160901,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
5,CS021314000093,江藤 美佐,1,女性,1986-06-03,32,259-1113,神奈川県伊勢原市粟窪**********,S14021,20151015,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
6,CS021413000049,大野 幸子,1,女性,1973-04-17,45,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150703,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,1.394409
7,CS021313000025,砂川 あさみ,1,女性,1981-10-05,37,259-1131,神奈川県伊勢原市伊勢原**********,S14021,20150326,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,0.474282
8,CS021413000022,布施 花,1,女性,1972-05-18,46,259-1131,神奈川県伊勢原市伊勢原**********,S14021,20150225,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,0.474282
9,CS021413000094,宇佐美 メイサ,1,女性,1970-05-18,48,259-1131,神奈川県伊勢原市伊勢原**********,S14021,20150422,...,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0,0.474282


---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
df = df_receipt.copy()
df1 = df.groupby('customer_id').amount.sum().reset_index().rename(columns={'amount':'sum_amount'})
merge = pd.merge(df_customer,df1, how='left', on= 'customer_id')
merge['sum_amount'] = merge['sum_amount'].fillna(0)
merge.sort_values(['sum_amount', 'customer_id'], ascending=[False, True], inplace=True)
#昇順に並び変えてkeep=firstを指定することによって
merge.drop_duplicates(subset=['customer_name', 'postal_cd'],keep='first', inplace = True)
print('顧客数', len(df_customer),
      '名寄せ', len(merge),
      'diff', len(df_customer) - len(merge))

顧客数 21971 名寄せ 21941 diff 30


---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
>
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [ ]:
result = pd.merge(df_customer, merge[['customer_name', 'postal_cd', 'customer_id']], how='inner',on =['customer_name', 'postal_cd'])
result.rename(columns={'customer_id_x':'customer_id', 'customer_id_y':'dub'}, inplace=True)
print('diff', len(result['customer_id']) - len(result['dub'].unique()))

diff 30


---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
>
> ファイル出力先のパスは以下のようにすること
>
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
>
> ファイル出力先のパスは以下のようにすること。
>
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
>
> ファイル出力先のパスは以下のようにすること。
>
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
>
> ファイル出力先のパスは以下のようにすること
>
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！